In [ ]:
# Import necessary libraries
import os  # Provides a way of using operating system dependent functionality
import numpy as np  # Adds support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
from PIL import Image  # Opens and manipulates images
import tensorflow as tf  # Provides a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML, and developers easily build and deploy ML powered applications
from sklearn.svm import OneClassSVM  # Imports OneClassSVM, a tool for anomaly detection
import joblib  # Provides utilities for saving and loading Python objects that make use of NumPy data structures
from google.colab import files  # Used for file upload/download in Google Colab
import zipfile  # Allows reading and writing of ZIP files

# Upload a ZIP file and extract its contents
uploaded = files.upload()  # Open a file dialog for the user to select a ZIP file
zip_file_name = next(iter(uploaded))  # Get the name of the uploaded file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:  # Open the uploaded ZIP file
    zip_ref.extractall('images')  # Extract its contents into the 'images' directory

# Initialize the MobileNetV2 model excluding the top layer and add a global average pooling layer
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Load MobileNetV2 pre-trained on ImageNet, excluding the top classification layer
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()  # Add a global average pooling layer to reduce the feature map size
model = tf.keras.Sequential([
    base_model,
    global_average_layer
])

# Function to process and preprocess an image
def process_image(image_path):
    image = Image.open(image_path)  # Open the image from the given path
    if image.mode != 'RGB':
        image = image.convert('RGB')  # Convert the image to RGB if it is not
    image_resized = image.resize((224, 224))  # Resize the image to 224x224
    image_array = np.array(image_resized)  # Convert the image to a numpy array
    image_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)  # Preprocess the image using MobileNetV2's preprocessing
    return image_preprocessed

# Function to extract features from images in the given directory
def extract_features(data_dir='images'):
    features = []  # Initialize a list to store features
    valid_extensions = ['.jpg', '.jpeg', '.png', '.tif']  # Define valid image file extensions
    for root, _, files in os.walk(data_dir):  # Traverse the directory
        for file in files:
            if file.lower().endswith(tuple(valid_extensions)):  # Check if the file is an image
                img_path = os.path.join(root, file)  # Get the full path of the image
                print(f"Processing image: {img_path}")
                processed_img = process_image(img_path)  # Process and preprocess the image
                feature = model.predict(np.expand_dims(processed_img, axis=0)).flatten()  # Predict the features using the MobileNetV2 model
                features.append(feature)  # Append the features to the list
    return np.array(features)  # Return the features as a numpy array

# Extract features from images in the 'images' directory
features = extract_features()

# Train a model using OneClassSVM
# Note: OneClassSVM is used for anomaly detection and does not require labels
classifier = OneClassSVM(gamma='auto')  # Initialize OneClassSVM with 'auto' gamma value, which can be adjusted along with other parameters
classifier.fit(features)  # Train the classifier on the extracted features

# Save and download the model
classifier_model_filename = 'classifier_model.pkl'  # Define the filename for the saved model
joblib.dump(classifier, classifier_model_filename)  # Save the trained classifier to a file
files.download(classifier_model_filename)  # Download the model file to the local system


Saving Mehl-595.zip to Mehl-595.zip
9406464/9406464 [==============================] - 0s 0us/step
Processing image: images/Mehl-595/9653_10_561_2021-08-03_13-36-16_albedo.tif
1/1 [==============================] - 1s 699ms/step
Processing image: images/Mehl-595/9653_10_246_2021-08-03_08-32-13_albedo.tif
1/1 [==============================] - 0s 36ms/step
Processing image: images/Mehl-595/8818_03_2_2021-03-09_10-40-19_normal.tif
1/1 [==============================] - 0s 32ms/step
Processing image: images/Mehl-595/8818_03_11_2021-03-09_11-17-47_normal.tif
1/1 [==============================] - 0s 36ms/step
Processing image: images/Mehl-595/9653_10_255_2021-08-03_09-15-39_normal.tif
1/1 [==============================] - 0s 33ms/step
Processing image: images/Mehl-595/8818_04_35_2021-03-10_11-29-14_normal.tif
1/1 [==============================] - 0s 35ms/step
Processing image: images/Mehl-595/9653_10_184_2021-08-02_09-58-27_albedo.tif
1/1 [==============================] - 0s 40ms/step
P

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>